In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import col, SparkContext, udf
from pyspark.sql.types import *
import numpy as np

In [2]:
appName = "Big Data ML"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
# spark = sqlContext.sparkSession.builder.getOrCreate()
spark = sqlContext.sparkSession.builder.getOrCreate()

C:\Users\Woodw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
mqtt_raw = spark.read.csv("train70_augmented.csv",header=True, inferSchema= True)
mqtt_test_raw = spark.read.csv("test30_augmented.csv",header=True, inferSchema= True)

# Rename columns
for column_name in mqtt_raw.columns:
    new_column_name = column_name.replace(".", "_")
    mqtt_raw = mqtt_raw.withColumnRenamed(column_name, new_column_name)

for column_name in mqtt_test_raw.columns:
    new_column_name = column_name.replace(".", "_")
    mqtt_test_raw = mqtt_test_raw.withColumnRenamed(column_name, new_column_name)

# Drop the few rows with NA values (only 190)
mqtt_raw = mqtt_raw.na.drop()
mqtt_test_raw = mqtt_test_raw.na.drop()

# Dropping a couple useless columns
mqtt_raw = mqtt_raw.drop("mqtt_protoname")
mqtt_test_raw = mqtt_test_raw.drop("mqtt_protoname")

# Converting nominal columns to string so they can be encoded by the transformer 
# mqtt_raw = mqtt_raw.withColumn("mqtt_msgtype", mqtt_raw["mqtt_msgtype"].cast("string"))
# mqtt_raw = mqtt_raw.withColumn("mqtt_conack_val", mqtt_raw["mqtt_conack_val"].cast("string"))
# mqtt_test_raw = mqtt_test_raw.withColumn("mqtt_msgtype", mqtt_test_raw["mqtt_msgtype"].cast("string"))
# mqtt_test_raw = mqtt_test_raw.withColumn("mqtt_conack_val", mqtt_test_raw["mqtt_conack_val"].cast("string"))

In [4]:
mqtt_raw.printSchema()

root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-

In [5]:
col_names =  ["tcp_flags","tcp_time_delta","tcp_len",
"mqtt_conack_flags","mqtt_conack_flags_reserved","mqtt_conack_flags_sp",
"mqtt_conack_val","mqtt_conflag_cleansess","mqtt_conflag_passwd",
"mqtt_conflag_qos","mqtt_conflag_reserved","mqtt_conflag_retain",
"mqtt_conflag_uname","mqtt_conflag_willflag",
"mqtt_conflags","mqtt_dupflag","mqtt_hdrflags",
"mqtt_kalive","mqtt_len","mqtt_msg","mqtt_msgid","mqtt_msgtype",
"mqtt_proto_len","mqtt_qos",
"mqtt_retain","mqtt_sub_qos","mqtt_suback_qos","mqtt_ver",
"mqtt_willmsg","mqtt_willmsg_len","mqtt_willtopic","mqtt_willtopic_len",
"target"]

nominal_cols = ["mqtt_msgtype", "mqtt_conack_val", "tcp_flags", "mqtt_hdrflags"]
binary_cols = ["mqtt_dupflag"]
continuous_cols = ["tcp_time_delta","tcp_len",
"mqtt_conack_flags_reserved","mqtt_conack_flags_sp",
"mqtt_conflag_cleansess","mqtt_conflag_passwd",
"mqtt_conflag_qos","mqtt_conflag_reserved","mqtt_conflag_retain",
"mqtt_conflag_uname","mqtt_conflag_willflag",
"mqtt_kalive","mqtt_len","mqtt_msg","mqtt_msgid",
"mqtt_proto_len","mqtt_qos",
"mqtt_retain","mqtt_sub_qos","mqtt_suback_qos","mqtt_ver",
"mqtt_willmsg","mqtt_willmsg_len","mqtt_willtopic","mqtt_willtopic_len",
"mqtt_conack_flags", "mqtt_conflags"]

In [6]:
#Not sure if we should drop "mqtt_hdrflags"
columns_to_drop = ["mqtt_msg", "mqtt_conack_flags", "mqtt_conflags", 
"mqtt_willtopic_len", "mqtt_willtopic", "mqtt_willmsg_len", "mqtt_willmsg",
"mqtt_suback_qos", "mqtt_sub_qos", "mqtt_conflag_willflag", "mqtt_conflag_retain",
"mqtt_conflag_reserved", "mqtt_conflag_qos", "mqtt_conack_flags_sp", 
"mqtt_conack_flags_reserved"] # Add any columns that we want to just get rid of here

In [7]:
nominal_id_cols = [x+"_index" for x in nominal_cols]
nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
stage_nominal_indexer = StringIndexer(inputCols=nominal_cols, outputCols=nominal_id_cols)
# Stage where the index columns are further transformed using OneHotEncoder
stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

indexer_model = stage_nominal_indexer.fit(mqtt_raw)
mqtt_raw_indx = indexer_model.transform(mqtt_raw)
encoder_model = stage_nominal_onehot_encoder.fit(mqtt_raw_indx)
mqtt_encoded = encoder_model.transform(mqtt_raw_indx)

In [8]:
c = "mqtt_hdrflags"
mqtt_raw_indx.select(c+"_index").distinct().show()
mqtt_encoded.select(c+"_encoded").distinct().show()

+-------------------+
|mqtt_hdrflags_index|
+-------------------+
|                8.0|
|                0.0|
|                7.0|
|                1.0|
|                4.0|
|               11.0|
|                3.0|
|                2.0|
|               10.0|
|               13.0|
|                6.0|
|                5.0|
|                9.0|
|               12.0|
+-------------------+

+---------------------+
|mqtt_hdrflags_encoded|
+---------------------+
|           (13,[],[])|
|       (13,[4],[1.0])|
|       (13,[2],[1.0])|
|       (13,[6],[1.0])|
|      (13,[11],[1.0])|
|       (13,[7],[1.0])|
|      (13,[12],[1.0])|
|       (13,[5],[1.0])|
|       (13,[9],[1.0])|
|       (13,[0],[1.0])|
|       (13,[1],[1.0])|
|       (13,[3],[1.0])|
|       (13,[8],[1.0])|
|      (13,[10],[1.0])|
+---------------------+



In [9]:
mqtt_raw.show(1,vertical=True)

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000018 
 tcp_time_delta             | 1.000039   
 tcp_len                    | 13         
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0x00000030 
 mqtt_kalive                | 0.0        
 mqtt_len                   | 11.0       
 mqtt_msg                   | 32         
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 3.0        
 mqtt_proto_len             | 0.0 

In [10]:
from pyspark.sql.functions import when, isnan
nan_col = [col(column) for column in mqtt_raw.columns]
nan_val = mqtt_raw.select(*nan_col).select(*[
    when(isnan(col(c)), c).alias(c) for c in mqtt_raw.columns
])

print(nan_val.count())
print(mqtt_raw.count())

14000000
14000000


In [11]:
'''
Data preprocessing pipeline
'''
class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    def __init__(self):
        super().__init__()

    def label_to_vector(self, name):
        name = name.lower()
        
        if name == 'legitimate':
            return 0.0
        elif name == 'dos':
            return 1.0
        elif name == 'malformed':
            return 2.0
        elif name == 'slowite':
            return 3.0
        elif name == 'bruteforce':
            return 4.0
        elif name == 'flood':
            return 5.0
        else:
            return -100.0

    def _transform(self, dataset):
        label_to_vector_udf = udf(self.label_to_vector, StringType())
        output_df = dataset.withColumn('outcome', label_to_vector_udf(col('target'))).drop("target")  
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self, targetCols):
        super().__init__()
        self.target_cols = targetCols

    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.target_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df

class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    impute_columns = binary_cols+continuous_cols
    for col_name in columns_to_drop:
        if col_name in impute_columns:
            impute_columns.remove(col_name)
    stage_typecaster = FeatureTypeCaster(impute_columns)

    # Stage where coulmns are imputed if they have NAs
    stage_imputer = Imputer(inputCols=impute_columns, outputCols=impute_columns)

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols=nominal_cols, outputCols=nominal_id_cols)

    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = impute_columns+nominal_onehot_cols
    for col_name in columns_to_drop:
        if col_name in nominal_cols:
            feature_cols.remove(col_name+"_encoded")
            feature_cols.remove(col_name+"_index")
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columbs, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_cols + nominal_id_cols + nominal_onehot_cols +
        binary_cols + continuous_cols + columns_to_drop + ['vectorized_features'])
    # stage_column_dropper = ColumnDropper(columns_to_drop = nominal_id_cols + columns_to_drop + ['vectorized_features'])
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_typecaster, stage_imputer, stage_nominal_indexer, stage_nominal_onehot_encoder,
        stage_vector_assembler, stage_scaler, stage_outcome, stage_column_dropper])
    return pipeline

In [12]:
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(mqtt_raw)

mqtt_df = preprocess_pipeline_model.transform(mqtt_raw)
mqtt_df_test = preprocess_pipeline_model.transform(mqtt_test_raw)

In [13]:
from pyspark.sql.functions import when, isnan
nan_col = [col(column) for column in mqtt_raw.columns]
nan_val = mqtt_raw.select(*nan_col).select(*[
    when(isnan(col(c)), c).alias(c) for c in mqtt_raw.columns
])

nan_val.show(vertical=True)

-RECORD 0--------------------------
 tcp_flags                  | null 
 tcp_time_delta             | null 
 tcp_len                    | null 
 mqtt_conack_flags          | null 
 mqtt_conack_flags_reserved | null 
 mqtt_conack_flags_sp       | null 
 mqtt_conack_val            | null 
 mqtt_conflag_cleansess     | null 
 mqtt_conflag_passwd        | null 
 mqtt_conflag_qos           | null 
 mqtt_conflag_reserved      | null 
 mqtt_conflag_retain        | null 
 mqtt_conflag_uname         | null 
 mqtt_conflag_willflag      | null 
 mqtt_conflags              | null 
 mqtt_dupflag               | null 
 mqtt_hdrflags              | null 
 mqtt_kalive                | null 
 mqtt_len                   | null 
 mqtt_msg                   | null 
 mqtt_msgid                 | null 
 mqtt_msgtype               | null 
 mqtt_proto_len             | null 
 mqtt_qos                   | null 
 mqtt_retain                | null 
 mqtt_sub_qos               | null 
 mqtt_suback_qos            

In [14]:
mqtt_df_test.printSchema()

root
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)



In [15]:
mqtt_df_test.select("features").show(2, vertical=True, truncate=False)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 features | (44,[0,2,7,8,10,14,23,25,37],[8.406366363108786,0.2326203070881879,4.537380890548288,5.407040445080942,4.997138436817394,2.0308679449405815,10.07892593970219,2.0504165034845894,8.406366363108786]) 
-RECORD 1--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 features | (44,[1,13,23,25,31],[1.0984566854049087E-6,2.000421325597082,10.07892593970219,2.0504165034845894,2.000421325597082])                                                                                
only showing top 2 rows



In [16]:
mqtt_df_test.show(2, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 features | (44,[0,2,7,8,10,14,23,25,37],[8.406366363108786,0.2326203070881879,4.537380890548288,5.407040445080942,4.997138436817394,2.0308679449405815,10.07892593970219,2.0504165034845894,8.406366363108786]) 
 outcome  | 1.0                                                                                                                                                                                                  
-RECORD 1--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 features | (44,[1,13,23,25,31],[1.0984566854049087E-6,2.000421325597082,10.07892593970219,2.0504165034845894,2.000421325597082])                               

In [17]:
#Model 1 - Logistic Regression
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

lr = LogisticRegression(featuresCol = 'features', labelCol = 'outcome')
lrModel = lr.fit(mqtt_df) 

In [18]:
from pyspark.ml.classification import RandomForestClassifier

rfc = RandomForestClassifier(featuresCol = 'features', labelCol = 'outcome')
rf_model = rfc.fit(mqtt_df)

In [19]:
pred_LR = lrModel.transform(mqtt_df_test)
y_pred = pred_LR.select('outcome').collect()
y = mqtt_df_test.select('outcome').collect()
correct = 0
for i in range(len(y)):
    if y[i][0]==y_pred[i][0]:
        correct += 1
print("LR Test Accuracy:",correct/len(y)*100, "%")

LR Test Accuracy: 100.0 %


In [20]:
pred_DT = rf_model.transform(mqtt_df_test)
y_pred_DT = pred_DT.select('outcome').collect()
correct = 0
for i in range(len(y)):
    if y[i][0]==y_pred_DT[i][0]:
        correct += 1
print("DT Test Accuracty:",correct/len(y)*100, "%")

DT Test Accuracty: 100.0 %


Pytorch Learning 

In [ ]:
mqtt_df_val, mqtt_df_test = mqtt_df_test.randomSplit([0.5, 0.5])